In [ ]:
import pandas as pd
import arviz as az
from pathlib import Path
from matplotlib import pyplot as plt

from estival.sampling import tools as esamptools

import utils as ut

In [ ]:
analysis_name = "2024_07_30@13_55_32_test"

### Load idatas

In [ ]:
idatas_folder = Path.cwd() / "outputs" / analysis_name / "idatas"
idatas = {}
for file_path in idatas_folder.iterdir():
    fixed_param = str(file_path).split("idata_")[1].split(".nc")[0]
    if fixed_param == "None":
        fixed_param = None
    idatas[fixed_param] = az.from_netcdf(file_path)

### Check maximum Rhats and plot traces

In [ ]:
for fixed_param, idata in idatas.items():
    max_rhat = float(az.rhat(idatas[fixed_param]).to_array().max())
    buffer = "" if max_rhat > 1.1 else "           "
    print(f"{buffer}{fixed_param}: {max_rhat}")
    # az.plot_trace(idatas[fixed_param])

### Load full runs and check model fits

In [ ]:
full_runs_folder = Path.cwd() / "outputs" / analysis_name / "full_runs"

for fixed_param, idata in idatas.items(): 
    full_runs = pd.read_parquet(full_runs_folder / f"fullruns_{fixed_param}_None.parquet")
    esamptools.quantiles_for_results(full_runs, [0.025,0.25,0.5,0.75,0.975])["tb_prevalence_per100k"].loc[2010:].plot()
    ut.target_data.loc[2010:].plot(style='.',color='red') 
    plt.title(f"Fixed param: {fixed_param}")